In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

In [5]:
# Load the cleaned dataset
file_path = 'cleaned_size_chart_data.csv'
data = pd.read_csv(file_path)
data.head()

,Height (cm),Weight (kg),Chest (cm),Waist (cm),Hip (cm),Size Purchased,Size Exchanged
0,188,74,82,87,90,S,XL
1,164,67,82,76,90,L,S
2,157,60,104,68,102,XS,M
3,170,78,84,80,106,XL,S
4,188,66,86,90,87,XS,M


In [6]:
# Encode categorical columns
label_encoder = LabelEncoder()
data['Size Purchased'] = label_encoder.fit_transform(data['Size Purchased'])
data['Size Exchanged'] = label_encoder.fit_transform(data['Size Exchanged'])
data.head()

,Height (cm),Weight (kg),Chest (cm),Waist (cm),Hip (cm),Size Purchased,Size Exchanged
0,188,74,82,87,90,2,3
1,164,67,82,76,90,0,2
2,157,60,104,68,102,4,1
3,170,78,84,80,106,3,2
4,188,66,86,90,87,4,1


In [7]:
label_encoder.classes_

array(['L', 'M', 'S', 'XL'], dtype=object)

In [8]:
# Define features and target
X = data[['Height (cm)', 'Weight (kg)', 'Chest (cm)', 'Waist (cm)', 'Hip (cm)', 'Size Purchased']]
y = data['Size Exchanged']

# Split data for training and evaluation (though we use full data for final model training)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
# Scale the features for SVM
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
# Train the SVM model
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_scaled, y_train)


SVC(kernel='linear', random_state=42)

In [11]:
# Evaluate the model (optional, on test split)
y_test_pred = svm_model.predict(X_test_scaled)
print("Testing Accuracy:", accuracy_score(y_test, y_test_pred))
print("Classification Report:\n", classification_report(y_test, y_test_pred))

Testing Accuracy: 0.378
Classification Report:
               precision    recall  f1-score   support

           0       0.39      0.60      0.47       601
           1       0.00      0.00      0.00       396
           2       0.38      0.37      0.38       505
           3       0.36      0.41      0.39       498

    accuracy                           0.38      2000
   macro avg       0.28      0.35      0.31      2000
weighted avg       0.30      0.38      0.33      2000



c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
# Train on the full dataset now that we know it performs reasonably
X_scaled = scaler.fit_transform(X)
svm_model.fit(X_scaled, y)

SVC(kernel='linear', random_state=42)

In [ ]:
#Saving the model
import pickle

with open('app/model/apparel_size_predictor_model.pkl', 'wb') as model_file:
    pickle.dump(svm_model, model_file)
print("Model saved as a .pkl file.")


Model saved as a .pkl file.


In [ ]:
#saving Label Encoder
import pickle

# Save the LabelEncoder mappings
with open('app/model/label_encoder_mapping.pkl', 'wb') as le_file:
    pickle.dump(label_encoder.classes_, le_file)


In [25]:
# Function to get user input and predict the "Size Exchanged"
def predict_size_exchanged(height, weight, chest, waist, hip, size_purchased):
    # Create a dataframe from user input
    user_data = pd.DataFrame({
        'Height (cm)': [height],
        'Weight (kg)': [weight],
        'Chest (cm)': [chest],
        'Waist (cm)': [waist],
        'Hip (cm)': [hip],
        'Size Purchased': [size_purchased]
    })

    # Scale the user data
    user_data_scaled = scaler.transform(user_data)
    
    # Predict the size exchanged
    predicted_size = svm_model.predict(user_data_scaled)
    return predicted_size[0]


In [26]:
classes = label_encoder.classes_
classes

array(['L', 'M', 'S', 'XL'], dtype=object)

In [52]:
# Example usage with user input
user_height = int(input("Enter Height (cm)"))
user_weight = int(input("Enter Weight (kg): "))
user_chest = int(input("Enter Chest (cm): "))
user_waist = int(input("Enter Waist (cm): "))
user_hip = int(input("Enter Hip (cm): "))
user_size_purchased = input(f"Enter size purchased (e.g.{classes}): ")
user_size_purchased = label_encoder.transform([user_size_purchased])[0]
user_size_purchased

0

In [53]:
predicted_size = predict_size_exchanged(height=user_height, weight=user_weight, chest=user_chest, waist=user_waist, hip=user_hip, size_purchased=3)
predicted_size = label_encoder.classes_[predicted_size]
print(f"Recommended Size: {predicted_size}")

Recommended Size: L
